In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

# 生成数据
num_samples = 1000
user_ability = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)
problem_difficulty = np.clip(np.random.normal(500, 150, num_samples), 0, 1000)
data = pd.DataFrame({
    'user_ability': user_ability,
    'problem_difficulty': problem_difficulty,
    'accuracy': [(ua / 1000) * (1 - (pd / 1000)) > 0.5 for ua, pd in zip(user_ability, problem_difficulty)]
})


In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:



X = data[['user_ability', 'problem_difficulty']]
y = data['accuracy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

In [5]:
model = Sequential([
    Input(shape=(2,)),  # 使用Input层来定义输入形状
    Dense(64, activation='relu'),  # 第一层不再需要input_dim
    Dense(64, activation='relu'),  # 第二层密集层
    Dense(1, activation='sigmoid')  # 输出层
])

### 模型解释
#### Sequential 模型：这是一种线性堆叠的模型，可以通过简单地添加层来构建模型。
#### Dense 层：全连接层，第一个参数是神经元的数量，activation 是激活函数。
### 激活函数：
#### ReLU：非线性函数，用于添加非线性因素，帮助网络学习复杂的模式。
#### Sigmoid：将输出压缩到0和1之间，适用于二分类。
#### 优化器（Adam）：一种效果很好的随机梯度下降方法。
#### 损失函数（binary_crossentropy）：适用于二分类问题的损失函数。
###训练和评估
#### fit 方法用于训练模型，epochs 控制训练轮数，batch_size 控制每次训练的样本数量。
#### evaluate 方法用于在测试数据上评估模型的效果。
这就是使用TensorFlow构建和训练一个简单神经网络模型的基本过程。你可以根据具体的数据和需求调整模型的架构和参数。

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7931 - loss: 8.0246       
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.9734 - loss: 0.3332 
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.9688 - loss: 0.2443 
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accuracy: 0.9659 - loss: 0.1874 
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - accuracy: 0.9782 - loss: 0.1753   
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.9768 - loss: 0.2033   
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.9768 - loss: 0.2069 
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - accuracy: 0.9768 - loss: 0.2122 
Epoch 9/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.9768 - loss: 0.2187 
Epoch 10/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.9768 - loss: 0.2312 
Epoch 11/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - accuracy: 0.9768 - loss: 0.2215 
Epoch 12/50
80/80 ━━━━━━━━━━━━━━━

In [7]:
class BinaryClassifier:
    def __init__(self, input_dim, learning_rate=0.001, reg_lambda=0.001):
        self.model = self.build_model(input_dim, learning_rate, reg_lambda)

    def build_model(self, input_dim, learning_rate, reg_lambda):
        model = tf.keras.Sequential(
            [
                tf.keras.layers.Input(shape=(input_dim,)),
                tf.keras.layers.Dense(
                    128,
                    activation="relu",
                    kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                ),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(
                    64,
                    activation="relu",
                    kernel_regularizer=tf.keras.regularizers.l2(reg_lambda),
                ),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation="sigmoid"),
            ]
        )
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
        )
        return model

    def train(self, X_train, y_train, epochs=100, batch_size=32):
        self.model.fit(
            X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1
        )

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        return loss, accuracy

    def predict(self, X):
        return self.model.predict(X)


In [8]:
classifier = BinaryClassifier(input_dim=2)

# 训练模型
classifier.train(X_train, y_train)

# 评估模型
loss, accuracy = classifier.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9129 - loss: 8.1086
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.9568 - loss: 5.2698
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.9435 - loss: 4.4634
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9550 - loss: 4.0474 
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9561 - loss: 2.6147 
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9622 - loss: 2.0644
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.9616 - loss: 1.3051
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.9595 - loss: 1.1893
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9656 - loss: 1.1057 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9597 - loss: 0.8541 
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.9664 - loss: 1.0779
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 644u